In [6]:
import pymongo
from dotenv import load_dotenv
import os
from dataclasses import dataclass
load_dotenv()
import json
import hashlib
import datetime

In [7]:
# Connect to MongoDB
client = pymongo.MongoClient(os.getenv("MONGO_URI"))
res_collection = client['beschlusswiki']['resolutions']


# Get all resolutions from the database
resolutions = res_collection.find()

for resolution in resolutions:
    print("Processing resolution: " + resolution['rid'])
    res = resolution.copy()
    body = res["body"]
    applicants = [applicant for applicant in body["applicants"]]
    category = body["category"]

    stringified = (res['rid'] + res["rcode"] + res["user"] + body["title"] +
                    body["tag"] + "".join(applicants) + str(body["year"]) +
                    category["name"] + category["id"] + body["text"])    

    stringified = stringified.encode('utf-8')
    # Create sha256 hash
    hash = hashlib.sha256(stringified).hexdigest()

    # Add hash to resolution
    resolution['hash'] = hash
    # Update resolution in database
    res_collection.update_one({'_id': resolution['_id']}, {"$set": resolution}, upsert=False)

Processing resolution: 12ab
